This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = '6ytA_agFWmtsPou6zQT9'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?&start_date=2019-01-07&end_date=2019-01-09&api_key="
+API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2019-01-07', 'end_date': '2019-01-09', 'frequency': 'daily', 'data': [['2019-01-09', None, 76.55, 74.55, 74.8, None, None, None, None, None, None], ['2019-01-08', None, 74.95, 72.5, 74.2, None, None, None, None, None, None], ['2019-01-07', None, 72.65, 70.9, 72.6, None, None, None, None, None, None]], 'collapse': None, 'order': None}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

#### 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [7]:
r = requests.get("https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?&start_date=2017-01-01&end_date=2017-12-31&api_key="
+API_KEY)

In [8]:
#save the json file into a variable
afx_x_data = r.json()

#### 2. Convert the returned JSON object into a Python dictionary.

Since the date is unique, it is used as the key for the dictionary.

In [5]:
# The data is in reverse chronological order, and keeping the order is important
# for subsequent calculations
from collections import OrderedDict

In [9]:
#create dictionary
afx_x_dict = OrderedDict()
data = afx_x_data['dataset_data']['data']
column_names = afx_x_data['dataset_data']['column_names']
for d in data:
    afx_x_dict[d[0]] = {column_names[i]: d[i] for i in range(1,len(column_names))}

Examine the data structure for one of the dates (2017-12-29):

In [10]:
afx_x_dict['2017-12-29']

{'Change': None,
 'Close': 51.76,
 'Daily Traded Units': None,
 'Daily Turnover': None,
 'High': 51.94,
 'Last Price of the Day': None,
 'Low': 51.45,
 'Open': 51.76,
 'Traded Volume': 34640.0,
 'Turnover': 1792304.0}

afx_x_dict has a nested structure with date as the key, and other information in a dictionary. 

#### 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [15]:
#First get the highest opening price
highest_open = 0
highest_date = ''
for k, v in afx_x_dict.items():
    if v['Open'] is not None and v['Open'] > highest_open:
        highest_open, highest_date = v['Open'], k
    
#Then get the lowest opening price
lowest_open = highest_open
lowest_Date = ''
for k, v in afx_x_dict.items():
    if v['Open'] is not None and v['Open'] < lowest_open:
        lowest_open, lowest_date = v['Open'], k

In [19]:
print(f'The maximum opening price was {highest_open} on {highest_date}.')
print(f'The minimum opening price was {lowest_open} on {lowest_date}.')

The maximum opening price was 53.11 on 2017-12-14.
The minimum opening price was 34.0 on 2017-01-24.


#### 4. What was the largest change in any one day (based on High and Low price)?

In [24]:
largest_change = 0
largest_change_date = ''
for k,v in afx_x_dict.items():
    if v['High'] - v['Low'] > largest_change:
        date, largest_change = k, v['High'] - v['Low']

In [27]:
print(f'The largest change in one day was {largest_change} on {date}.')

The largest change in one day was 2.8100000000000023 on 2017-05-11.


#### 5. What was the largest change between any two days (based on Closing Price)?

For simplicity, first create a list of tuples with date and closing price:

In [55]:
afx_x_list = [(k,v['Close']) for k,v in afx_x_dict.items()]

Examine the first 5 elements:

In [57]:
afx_x_list[0:5]

[('2017-12-29', 51.76),
 ('2017-12-28', 51.6),
 ('2017-12-27', 51.82),
 ('2017-12-22', 51.32),
 ('2017-12-21', 51.4)]

In [60]:
largest_pos_change = 0
largest_neg_change = 0
for i in range(0,len(afx_x_list)-1):
    if afx_x_list[i][1] - afx_x_list[i+1][1] >= 0 and \
    afx_x_list[i][1] - afx_x_list[i+1][1] > largest_pos_change:
        largest_pos_change = afx_x_list[i][1] - afx_x_list[i+1][1]
        date_pos_2, date_pos_1 = afx_x_list[i][0],afx_x_list[i+1][0]
    elif afx_x_list[i][1] - afx_x_list[i+1][1] < largest_neg_change:
        largest_neg_change = afx_x_list[i][1] - afx_x_list[i+1][1]
        date_neg_2, date_neg_1 = afx_x_list[i][0],afx_x_list[i+1][0]

In [71]:
if largest_pos_change >= -1 * largest_neg_change:
    print(f'The largest change is {largest_pos_change} between {date_pos_1} and {date_pos_2}.')
else:
    print(f'The largest change is {largest_neg_change} between {date_neg_1} and {date_neg_2}.')

The largest change is -2.559999999999995 between 2017-08-08 and 2017-08-09.


#### 6. What was the average daily trading volume during this year?

In [82]:
traded_volume = [v['Traded Volume'] for v in afx_x_dict.values()]
avg_volume = sum(traded_volume)/len(traded_volume)
print(f'The average daily trading volume during 2017 is {avg_volume}.')

The average daily trading volume during 2017 is 89124.33725490196.


#### 7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

First create a function for median:

In [135]:
def median_calc(l):
    l = sorted(l)
    if len(l) % 2 == 1:
        return l[int(len(l)/2)]
    else:
        return (l[int(len(l)/2)] + l[int(len(l)/2-1)])/2

In [134]:
print(f'The median traded volume in 2017 is {median_calc(traded_volume)}.')

The median traded volume in 2017 is 76286.0.
